In [39]:
import spacy
import pandas as pd
import re
import numpy as np
from numpy import dot 
from numpy.linalg import norm
from numpy import average
from gensim.models import word2vec
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity


try:
    nlp = spacy.load("pt_core_news_sm")
except:
    print("Modelo SpaCy não encontrado. Verifique a instalação.")

solicitacoes = [
    "Qual a extensão da frente de fogo atualmente",
    "Como está a direção do vento em relação à frente de fogo",
    "Há risco de a frente de fogo se expandir para áreas urbanas",
    "Quais são as equipes e recursos disponíveis para combater a frente de fogo",
    "Como estão as condições meteorológicas afetando a propagação da frente de fogo",
    "Quais são os planos de evacuação próximos à frente de fogo",
    "O que a análise de satélite mostra sobre o comportamento da frente de fogo",
    "Qual é a estratégia atual para conter a frente de fogo",
    "Há algum foco secundário sendo alimentado pela frente de fogo principal",
    "A frente de fogo já foi controlada em alguma área ou ainda está em expansão",
    "Quais medidas estão sendo tomadas para proteger as propriedades próximas à frente de fogo",
    "Qual a previsão para os próximos dias em relação ao comportamento da frente de fogo",
    "Existem recursos aéreos disponíveis para combater a frente de fogo",
    "Quais são os principais obstáculos para o controle da frente de fogo nesta região",
    "Como o terreno está influenciando a propagação da frente de fogo",
    "Qual é o índice de qualidade do ar nesta região",
    "Quais são os níveis de PM2.5 e PM10 atualmente",
    "Há previsão de piora na qualidade do ar devido aos ventos",
    "Como a qualidade do ar está afetando a saúde da população local",
    "Qual é o status da qualidade do ar em áreas de risco",
    "Quais medidas estão sendo tomadas para melhorar a qualidade do ar",
    "Como a poluição do ar está impactando os transportes públicos",
    "Qual é a concentração de ozônio na camada atmosférica nesta área",
    "Quais fontes estão contribuindo mais para a poluição do ar local",
    "Há alguma recomendação para pessoas com problemas respiratórios devido à qualidade do ar",
    "A qualidade do ar piorou nas últimas horas ou está estabilizada",
    "Como o clima está influenciando a qualidade do ar atualmente",
    "Há previsão de tempestades ou chuvas que possam melhorar a qualidade do ar",
    "Quais são as cidades mais afetadas pela poluição do ar no momento",
    "A qualidade do ar está dentro dos padrões recomendados pela OMS"
]

# Normalização
def pre_processar(texto):
    doc = nlp(texto.lower())
    tokens_limpos = []
    for token in doc:
        # Stop Words e Pontuação
        if not token.is_stop and not token.is_punct:
            # Lematização
            tokens_limpos.append(token.lemma_)
            
    # O Scikit-Learn espera uma string inteira, não uma lista:
    return " ".join(tokens_limpos)

# Aplicando a limpeza em todos os documentos por laço for item em solicitações
corpus_processado = [pre_processar(item) for item in solicitacoes]

# Numerando corpus_processado visualmente
print("\n--- Amostra dos Documentos Processados ---\n")
for i, item in enumerate(corpus_processado[:5]):
    print(f"Doc {i+1}: {item}")

# TF-IDF
vectorizer = TfidfVectorizer()
matriz_tfidf = vectorizer.fit_transform(corpus_processado)

# Criando índices dinâmicos para N documentos 
lista_indices = [f"Solicitação {i+1}" for i in range(len(solicitacoes))]

# Tabulação dos vetores TF-IDF por pandas
df_vetores = pd.DataFrame(
    matriz_tfidf.toarray(), 
    columns=vectorizer.get_feature_names_out(),
    index=lista_indices
)
print("\n--- Vetores (Amostra das primeiras 4 linhas e colunas) ---\n")
print(df_vetores.iloc[:4, :4].round(2))

# Calculo cosseno
matriz_similaridade = cosine_similarity(matriz_tfidf)

# Transformando em DataFrame para visualizar a Matriz Bonita
df_sim = pd.DataFrame(
    matriz_similaridade,
    index=lista_indices,
    columns=lista_indices
)



print("\n--- Matriz de Similaridade de Cosseno (Amostra 4x4) ---\n")
print(df_sim.iloc[:4, :4].round(2))

# Comparando Doc 1 (Fogo) com Doc 2 (Vento/Fogo)
sim_1_2 = df_sim.loc["Solicitação 1", "Solicitação 2"]
print(f"\n Similaridade entre Solicit. 1 e 2: {sim_1_2:.2f}")

# Comparando Doc 1 (Fogo) com Doc 16 (Qualidade do Ar)
sim_1_16 = df_sim.loc["Solicitação 1", "Solicitação 16"]
print(f"\n Similaridade entre Solicit. 1 (Fogo) e 16 (Ar): {sim_1_16:.2f}")



--- Amostra dos Documentos Processados ---

Doc 1: extensão frente fogo atualmente
Doc 2: direção vento frente fogo
Doc 3: haver risco frente fogo expandir área urbano
Doc 4: equipe recurso disponível combater frente fogo
Doc 5: condição meteorológico afetar propagação frente fogo

--- Vetores (Amostra das primeiras 4 linhas e colunas) ---

               afetar  algum  alimentar  análise
Solicitação 1     0.0    0.0        0.0      0.0
Solicitação 2     0.0    0.0        0.0      0.0
Solicitação 3     0.0    0.0        0.0      0.0
Solicitação 4     0.0    0.0        0.0      0.0

--- Matriz de Similaridade de Cosseno (Amostra 4x4) ---

               Solicitação 1  Solicitação 2  Solicitação 3  Solicitação 4
Solicitação 1           1.00           0.19           0.13           0.14
Solicitação 2           0.19           1.00           0.13           0.14
Solicitação 3           0.13           0.13           1.00           0.10
Solicitação 4           0.14           0.14           0.1

In [46]:
# Pre-processamento de W2V
def processar_texto_token(texto):
    doc = nlp(texto)
    lista_tokens = []
    for sentenca in doc.sents:
        tokens_frase = [token.lemma_.lower() for token in sentenca if not token.is_stop and not token.is_punct]
        lista_tokens.append(tokens_frase)
    return lista_tokens

tokens_w2v = []
for texto in solicitacoes:
    # processar_texto_token retorna uma LISTA DE FRASES (ex: 3 frases)
    frases_do_doc = processar_texto_token(texto)
    # .extend tira as frases da lista do documento e coloca na lista principal
    tokens_w2v.extend(frases_do_doc)


# 5. building the word2vec model
# Model configuration
feature_size = 32  # size of vector representation
window_context = 3
min_word_count = 1
sample = 1e-5
w2vec_model = word2vec.Word2Vec(tokens_w2v, vector_size= feature_size,
                                window=window_context, min_count= min_word_count,
                                sample=sample, epochs = 50)


v1 = w2vec_model.wv["fogo"]
v2 = w2vec_model.wv["frente"]
v3 = w2vec_model.wv["ar"]
print(v1)
print(v2)
print(v3)
print("\n cosseno de similaridade entre 'fogo' e 'frente': ")
print(dot(v1,v2)/(norm(v1)*norm(v2)))

print("\n cosseno de similaridade entre 'fogo' e 'ar': ")
print(dot(v1,v3)/(norm(v1)*norm(v3)))

[-0.00168282  0.00074303  0.0159577   0.02815004 -0.02906842 -0.02224352
  0.0201877   0.02803968 -0.01566963 -0.01175407  0.02306648 -0.00478121
 -0.01417607  0.02046872 -0.01518739 -0.00567633  0.0089874   0.00310884
 -0.02588558 -0.02952442  0.02285063  0.01584731  0.02112033  0.00238189
  0.01984761 -0.01064484 -0.00295414  0.01802679 -0.0234936  -0.01230363
 -0.02347484 -0.00290385]
[ 0.02980477 -0.02286406 -0.00728182 -0.00606051  0.02524471 -0.01854605
  0.00014987 -0.01485107 -0.03001124  0.01564878 -0.02735913 -0.01372091
 -0.00010466 -0.00093244 -0.02394107  0.03004561  0.0155712   0.02885665
 -0.02548664  0.01405046 -0.01292666  0.0025802   0.02656878 -0.0139433
  0.01412459 -0.02120513 -0.01108605  0.02938091 -0.0049213   0.00100466
 -0.01294461 -0.0240034 ]
[-0.00471253  0.00771811 -0.00277508  0.01729269 -0.0085718   0.0070627
  0.01704936  0.0260811  -0.00454294 -0.02877545  0.01365798  0.00178683
  0.02325596 -0.00254151 -0.00824504 -0.02735315 -0.00267674  0.00883301
 